In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [45]:
x = torch.randn(2, 8, 10)
x , x.shape

(tensor([[[-1.9169,  0.3649,  0.3007,  0.3399,  0.5696,  0.0340,  0.1653,
            2.0228,  1.6444, -0.0475],
          [ 0.1903, -0.3215,  2.0137, -1.0075, -1.0001, -2.0051, -0.2918,
            0.6211,  1.6006,  0.2178],
          [-0.3991,  1.1042, -0.2563, -0.4074,  1.6824, -2.3969,  0.3603,
            0.7626, -0.4506, -2.5584],
          [-0.4414, -1.0183, -0.9552, -1.3212,  0.3332, -1.8434, -1.3729,
           -2.2047,  0.5816,  0.2121],
          [-0.1442,  0.1927,  1.3300,  0.4236,  1.2717,  0.0567,  0.1549,
           -0.9081,  1.4621, -2.1316],
          [-0.2103, -0.4298, -2.1800,  0.5979, -2.5844, -0.6211,  0.6349,
            0.1723,  0.7373, -0.3392],
          [-0.9832,  0.6487, -0.7766,  0.0980, -1.2170,  0.6121, -0.7785,
            0.9321, -2.3463, -0.4403],
          [ 1.5136, -0.6235, -0.6906,  2.2320, -0.9410, -1.5890, -1.7063,
           -1.4288,  0.8631, -1.2873]],
 
         [[-0.1623, -1.6013, -0.1204,  0.7277,  0.1215,  0.2251, -0.2203,
            1.1314,

In [46]:
# 门控机制
gate = nn.Linear(10,4)
logits = gate(x)
logits,logits.shape

(tensor([[[ 0.2972, -0.7298, -0.5038,  0.8112],
          [-0.4873, -1.4042, -0.4193,  0.2624],
          [ 1.8845, -0.7369,  1.1626,  0.5766],
          [ 0.8990,  0.0150,  0.1176,  0.2054],
          [ 1.2475,  0.3019,  0.5251,  0.4485],
          [ 0.2727, -0.3511, -0.7550, -0.5606],
          [-0.6064, -0.9345,  0.0488, -0.6265],
          [ 1.5132,  0.5456, -0.3716, -1.1583]],
 
         [[-0.1888, -0.3492, -0.1754,  0.4613],
          [ 0.4196, -0.4732,  0.0711, -0.2526],
          [ 0.2858, -1.2856, -0.0434,  0.7420],
          [ 0.4729,  0.1665,  0.8551,  0.7604],
          [-0.9356, -1.5446, -0.2196,  0.6424],
          [-1.0974, -1.4197, -0.0784, -0.0178],
          [ 0.7034,  0.0416,  0.1377, -0.1695],
          [ 0.8113, -0.0300,  0.4306,  0.6282]]], grad_fn=<ViewBackward0>),
 torch.Size([2, 8, 4]))

In [48]:
# 激活两个专家
logits_topk, indices = logits.topk(2, dim=-1)
logits_topk,logits_topk.shape, indices,indices.shape

(tensor([[[ 0.8112,  0.2972],
          [ 0.2624, -0.4193],
          [ 1.8845,  1.1626],
          [ 0.8990,  0.2054],
          [ 1.2475,  0.5251],
          [ 0.2727, -0.3511],
          [ 0.0488, -0.6064],
          [ 1.5132,  0.5456]],
 
         [[ 0.4613, -0.1754],
          [ 0.4196,  0.0711],
          [ 0.7420,  0.2858],
          [ 0.8551,  0.7604],
          [ 0.6424, -0.2196],
          [-0.0178, -0.0784],
          [ 0.7034,  0.1377],
          [ 0.8113,  0.6282]]], grad_fn=<TopkBackward0>),
 torch.Size([2, 8, 2]),
 tensor([[[3, 0],
          [3, 2],
          [0, 2],
          [0, 3],
          [0, 2],
          [0, 1],
          [2, 0],
          [0, 1]],
 
         [[3, 2],
          [0, 2],
          [3, 0],
          [2, 3],
          [3, 2],
          [3, 2],
          [0, 2],
          [0, 3]]]),
 torch.Size([2, 8, 2]))

In [49]:
zeros = torch.full_like(logits, float("-inf"))
zeros,zeros.shape

(tensor([[[-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf]],
 
         [[-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf],
          [-inf, -inf, -inf, -inf]]]),
 torch.Size([2, 8, 4]))

In [50]:
sparse_logits = zeros.scatter(dim=-1, index=indices, src=logits_topk)
sparse_logits,sparse_logits.shape

(tensor([[[ 0.2972,    -inf,    -inf,  0.8112],
          [   -inf,    -inf, -0.4193,  0.2624],
          [ 1.8845,    -inf,  1.1626,    -inf],
          [ 0.8990,    -inf,    -inf,  0.2054],
          [ 1.2475,    -inf,  0.5251,    -inf],
          [ 0.2727, -0.3511,    -inf,    -inf],
          [-0.6064,    -inf,  0.0488,    -inf],
          [ 1.5132,  0.5456,    -inf,    -inf]],
 
         [[   -inf,    -inf, -0.1754,  0.4613],
          [ 0.4196,    -inf,  0.0711,    -inf],
          [ 0.2858,    -inf,    -inf,  0.7420],
          [   -inf,    -inf,  0.8551,  0.7604],
          [   -inf,    -inf, -0.2196,  0.6424],
          [   -inf,    -inf, -0.0784, -0.0178],
          [ 0.7034,    -inf,  0.1377,    -inf],
          [ 0.8113,    -inf,    -inf,  0.6282]]], grad_fn=<ScatterBackward0>),
 torch.Size([2, 8, 4]))

In [51]:
sparse_logits = F.softmax(sparse_logits, dim=-1)
sparse_logits,sparse_logits.shape

(tensor([[[0.3743, 0.0000, 0.0000, 0.6257],
          [0.0000, 0.0000, 0.3359, 0.6641],
          [0.6730, 0.0000, 0.3270, 0.0000],
          [0.6668, 0.0000, 0.0000, 0.3332],
          [0.6731, 0.0000, 0.3269, 0.0000],
          [0.6511, 0.3489, 0.0000, 0.0000],
          [0.3418, 0.0000, 0.6582, 0.0000],
          [0.7247, 0.2753, 0.0000, 0.0000]],
 
         [[0.0000, 0.0000, 0.3460, 0.6540],
          [0.5863, 0.0000, 0.4137, 0.0000],
          [0.3879, 0.0000, 0.0000, 0.6121],
          [0.0000, 0.0000, 0.5236, 0.4764],
          [0.0000, 0.0000, 0.2969, 0.7031],
          [0.0000, 0.0000, 0.4849, 0.5151],
          [0.6378, 0.0000, 0.3622, 0.0000],
          [0.5456, 0.0000, 0.0000, 0.4544]]], grad_fn=<SoftmaxBackward0>),
 torch.Size([2, 8, 4]))

In [52]:
gate_logit = logits.view(-1, 4)
gate_logit,gate_logit.shape

(tensor([[ 0.2972, -0.7298, -0.5038,  0.8112],
         [-0.4873, -1.4042, -0.4193,  0.2624],
         [ 1.8845, -0.7369,  1.1626,  0.5766],
         [ 0.8990,  0.0150,  0.1176,  0.2054],
         [ 1.2475,  0.3019,  0.5251,  0.4485],
         [ 0.2727, -0.3511, -0.7550, -0.5606],
         [-0.6064, -0.9345,  0.0488, -0.6265],
         [ 1.5132,  0.5456, -0.3716, -1.1583],
         [-0.1888, -0.3492, -0.1754,  0.4613],
         [ 0.4196, -0.4732,  0.0711, -0.2526],
         [ 0.2858, -1.2856, -0.0434,  0.7420],
         [ 0.4729,  0.1665,  0.8551,  0.7604],
         [-0.9356, -1.5446, -0.2196,  0.6424],
         [-1.0974, -1.4197, -0.0784, -0.0178],
         [ 0.7034,  0.0416,  0.1377, -0.1695],
         [ 0.8113, -0.0300,  0.4306,  0.6282]], grad_fn=<ViewBackward0>),
 torch.Size([16, 4]))

In [53]:
final_outputs = torch.zeros_like(x)
final_outputs,final_outputs.shape

(tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],
 
         [[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]]),
 torch.Size([2, 8, 10]))

In [54]:
# 展平
x_flat = x.view(-1, x.shape[-1])
x_flat,x_flat.shape,x.shape

(tensor([[-1.9169,  0.3649,  0.3007,  0.3399,  0.5696,  0.0340,  0.1653,  2.0228,
           1.6444, -0.0475],
         [ 0.1903, -0.3215,  2.0137, -1.0075, -1.0001, -2.0051, -0.2918,  0.6211,
           1.6006,  0.2178],
         [-0.3991,  1.1042, -0.2563, -0.4074,  1.6824, -2.3969,  0.3603,  0.7626,
          -0.4506, -2.5584],
         [-0.4414, -1.0183, -0.9552, -1.3212,  0.3332, -1.8434, -1.3729, -2.2047,
           0.5816,  0.2121],
         [-0.1442,  0.1927,  1.3300,  0.4236,  1.2717,  0.0567,  0.1549, -0.9081,
           1.4621, -2.1316],
         [-0.2103, -0.4298, -2.1800,  0.5979, -2.5844, -0.6211,  0.6349,  0.1723,
           0.7373, -0.3392],
         [-0.9832,  0.6487, -0.7766,  0.0980, -1.2170,  0.6121, -0.7785,  0.9321,
          -2.3463, -0.4403],
         [ 1.5136, -0.6235, -0.6906,  2.2320, -0.9410, -1.5890, -1.7063, -1.4288,
           0.8631, -1.2873],
         [-0.1623, -1.6013, -0.1204,  0.7277,  0.1215,  0.2251, -0.2203,  1.1314,
           0.3120, -0.2042],
 

In [55]:
sparse_logits_flat = sparse_logits.view(-1, sparse_logits.shape[-1])
sparse_logits_flat,sparse_logits_flat.shape,sparse_logits.shape

(tensor([[0.3743, 0.0000, 0.0000, 0.6257],
         [0.0000, 0.0000, 0.3359, 0.6641],
         [0.6730, 0.0000, 0.3270, 0.0000],
         [0.6668, 0.0000, 0.0000, 0.3332],
         [0.6731, 0.0000, 0.3269, 0.0000],
         [0.6511, 0.3489, 0.0000, 0.0000],
         [0.3418, 0.0000, 0.6582, 0.0000],
         [0.7247, 0.2753, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.3460, 0.6540],
         [0.5863, 0.0000, 0.4137, 0.0000],
         [0.3879, 0.0000, 0.0000, 0.6121],
         [0.0000, 0.0000, 0.5236, 0.4764],
         [0.0000, 0.0000, 0.2969, 0.7031],
         [0.0000, 0.0000, 0.4849, 0.5151],
         [0.6378, 0.0000, 0.3622, 0.0000],
         [0.5456, 0.0000, 0.0000, 0.4544]], grad_fn=<ViewBackward0>),
 torch.Size([16, 4]),
 torch.Size([2, 8, 4]))

In [56]:
expert_mask = (indices == 0)
indices,expert_mask,expert_mask.shape

(tensor([[[3, 0],
          [3, 2],
          [0, 2],
          [0, 3],
          [0, 2],
          [0, 1],
          [2, 0],
          [0, 1]],
 
         [[3, 2],
          [0, 2],
          [3, 0],
          [2, 3],
          [3, 2],
          [3, 2],
          [0, 2],
          [0, 3]]]),
 tensor([[[False,  True],
          [False, False],
          [ True, False],
          [ True, False],
          [ True, False],
          [ True, False],
          [False,  True],
          [ True, False]],
 
         [[False, False],
          [ True, False],
          [False,  True],
          [False, False],
          [False, False],
          [False, False],
          [ True, False],
          [ True, False]]]),
 torch.Size([2, 8, 2]))

In [57]:
# 每个token 是否用到当前i的专家
expert_mask = expert_mask.any(-1)
expert_mask ,expert_mask.shape

(tensor([[ True, False,  True,  True,  True,  True,  True,  True],
         [False,  True,  True, False, False, False,  True,  True]]),
 torch.Size([2, 8]))

In [58]:
#展平
expert_mask_flat = expert_mask.view(-1)
expert_mask_flat,expert_mask_flat.shape


(tensor([ True, False,  True,  True,  True,  True,  True,  True, False,  True,
          True, False, False, False,  True,  True]),
 torch.Size([16]))

In [59]:
# 取出当前专家需要计算的token
expert_input = x_flat[expert_mask_flat]
x_flat,x_flat.shape,expert_input,expert_input.shape

(tensor([[-1.9169,  0.3649,  0.3007,  0.3399,  0.5696,  0.0340,  0.1653,  2.0228,
           1.6444, -0.0475],
         [ 0.1903, -0.3215,  2.0137, -1.0075, -1.0001, -2.0051, -0.2918,  0.6211,
           1.6006,  0.2178],
         [-0.3991,  1.1042, -0.2563, -0.4074,  1.6824, -2.3969,  0.3603,  0.7626,
          -0.4506, -2.5584],
         [-0.4414, -1.0183, -0.9552, -1.3212,  0.3332, -1.8434, -1.3729, -2.2047,
           0.5816,  0.2121],
         [-0.1442,  0.1927,  1.3300,  0.4236,  1.2717,  0.0567,  0.1549, -0.9081,
           1.4621, -2.1316],
         [-0.2103, -0.4298, -2.1800,  0.5979, -2.5844, -0.6211,  0.6349,  0.1723,
           0.7373, -0.3392],
         [-0.9832,  0.6487, -0.7766,  0.0980, -1.2170,  0.6121, -0.7785,  0.9321,
          -2.3463, -0.4403],
         [ 1.5136, -0.6235, -0.6906,  2.2320, -0.9410, -1.5890, -1.7063, -1.4288,
           0.8631, -1.2873],
         [-0.1623, -1.6013, -0.1204,  0.7277,  0.1215,  0.2251, -0.2203,  1.1314,
           0.3120, -0.2042],
 

In [60]:
expert = nn.Linear(10, 10)

In [61]:
# 当前专家需要计算5个token
export_output = expert(expert_input)
export_output,export_output.shape

(tensor([[ 2.2763e-01, -6.6976e-01,  3.5573e-02,  7.3793e-02,  1.0876e+00,
          -5.6309e-01,  1.7597e-01, -4.7608e-01,  1.8169e-01, -3.3662e-01],
         [ 4.1032e-01, -5.3572e-01, -1.9546e+00, -3.2027e-01,  1.4034e+00,
          -1.2927e+00, -7.8958e-01, -8.3510e-01,  1.0911e+00, -9.6988e-01],
         [ 6.9326e-01,  3.3735e-01,  1.9023e-01,  1.0221e-01,  7.8824e-01,
           1.8935e-01,  1.8323e-01, -7.9960e-01,  6.3797e-01, -8.9355e-01],
         [ 2.7632e-01, -3.2140e-01, -6.2477e-02, -1.4784e-02,  1.0111e+00,
          -4.6016e-01,  1.4112e+00, -1.1115e-01,  1.0467e+00, -7.5915e-01],
         [-4.8984e-01, -2.6551e-01, -4.2119e-02, -9.8407e-01,  1.5780e-01,
           6.4319e-01, -1.0525e+00,  3.7018e-01,  9.5109e-01,  9.7007e-01],
         [-9.2653e-01, -3.2341e-01, -1.8357e-01, -9.9574e-01,  2.1067e-01,
          -5.2553e-01, -1.0058e+00,  4.8491e-01, -2.2569e-01, -1.9420e-01],
         [-8.2126e-01, -6.3031e-02, -8.7859e-02, -1.8947e-01,  9.9473e-01,
           1.1585e+

In [64]:
# 每个token对应专家分数或者权重
sparse_logits_flat,sparse_logits_flat.shape,expert_mask_flat

(tensor([[0.3743, 0.0000, 0.0000, 0.6257],
         [0.0000, 0.0000, 0.3359, 0.6641],
         [0.6730, 0.0000, 0.3270, 0.0000],
         [0.6668, 0.0000, 0.0000, 0.3332],
         [0.6731, 0.0000, 0.3269, 0.0000],
         [0.6511, 0.3489, 0.0000, 0.0000],
         [0.3418, 0.0000, 0.6582, 0.0000],
         [0.7247, 0.2753, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.3460, 0.6540],
         [0.5863, 0.0000, 0.4137, 0.0000],
         [0.3879, 0.0000, 0.0000, 0.6121],
         [0.0000, 0.0000, 0.5236, 0.4764],
         [0.0000, 0.0000, 0.2969, 0.7031],
         [0.0000, 0.0000, 0.4849, 0.5151],
         [0.6378, 0.0000, 0.3622, 0.0000],
         [0.5456, 0.0000, 0.0000, 0.4544]], grad_fn=<ViewBackward0>),
 torch.Size([16, 4]),
 tensor([ True, False,  True,  True,  True,  True,  True,  True, False,  True,
          True, False, False, False,  True,  True]))

In [65]:
# 提取门控分数
gate_scores = sparse_logits_flat[expert_mask_flat, 0]
gate_scores,gate_scores.shape

(tensor([0.3743, 0.6730, 0.6668, 0.6731, 0.6511, 0.3418, 0.7247, 0.5863, 0.3879,
         0.6378, 0.5456], grad_fn=<IndexBackward0>),
 torch.Size([11]))

In [66]:
gate_scores = gate_scores.unsqueeze(1)
gate_scores,gate_scores.shape

(tensor([[0.3743],
         [0.6730],
         [0.6668],
         [0.6731],
         [0.6511],
         [0.3418],
         [0.7247],
         [0.5863],
         [0.3879],
         [0.6378],
         [0.5456]], grad_fn=<UnsqueezeBackward0>),
 torch.Size([11, 1]))

In [67]:
weighted_output = export_output * gate_scores
export_output,weighted_output,weighted_output.shape

(tensor([[ 2.2763e-01, -6.6976e-01,  3.5573e-02,  7.3793e-02,  1.0876e+00,
          -5.6309e-01,  1.7597e-01, -4.7608e-01,  1.8169e-01, -3.3662e-01],
         [ 4.1032e-01, -5.3572e-01, -1.9546e+00, -3.2027e-01,  1.4034e+00,
          -1.2927e+00, -7.8958e-01, -8.3510e-01,  1.0911e+00, -9.6988e-01],
         [ 6.9326e-01,  3.3735e-01,  1.9023e-01,  1.0221e-01,  7.8824e-01,
           1.8935e-01,  1.8323e-01, -7.9960e-01,  6.3797e-01, -8.9355e-01],
         [ 2.7632e-01, -3.2140e-01, -6.2477e-02, -1.4784e-02,  1.0111e+00,
          -4.6016e-01,  1.4112e+00, -1.1115e-01,  1.0467e+00, -7.5915e-01],
         [-4.8984e-01, -2.6551e-01, -4.2119e-02, -9.8407e-01,  1.5780e-01,
           6.4319e-01, -1.0525e+00,  3.7018e-01,  9.5109e-01,  9.7007e-01],
         [-9.2653e-01, -3.2341e-01, -1.8357e-01, -9.9574e-01,  2.1067e-01,
          -5.2553e-01, -1.0058e+00,  4.8491e-01, -2.2569e-01, -1.9420e-01],
         [-8.2126e-01, -6.3031e-02, -8.7859e-02, -1.8947e-01,  9.9473e-01,
           1.1585e+

In [79]:
final_outputs[expert_mask].shape

torch.Size([11, 10])

In [77]:
final_outputs[expert_mask] += weighted_output
final_outputs,final_outputs.shape,expert_mask,expert_mask.shape

(tensor([[[ 2.5558e-01, -7.5201e-01,  3.9942e-02,  8.2855e-02,  1.2211e+00,
           -6.3224e-01,  1.9758e-01, -5.3455e-01,  2.0400e-01, -3.7796e-01],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
            0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
          [ 8.2849e-01, -1.0817e+00, -3.9466e+00, -6.4667e-01,  2.8336e+00,
           -2.6101e+00, -1.5943e+00, -1.6862e+00,  2.2031e+00, -1.9583e+00],
          [ 1.3867e+00,  6.7480e-01,  3.8051e-01,  2.0445e-01,  1.5767e+00,
            3.7876e-01,  3.6651e-01, -1.5994e+00,  1.2761e+00, -1.7874e+00],
          [ 5.5800e-01, -6.4903e-01, -1.2616e-01, -2.9855e-02,  2.0417e+00,
           -9.2924e-01,  2.8498e+00, -2.2445e-01,  2.1136e+00, -1.5330e+00],
          [-9.5677e-01, -5.1861e-01, -8.2268e-02, -1.9221e+00,  3.0821e-01,
            1.2563e+00, -2.0557e+00,  7.2304e-01,  1.8577e+00,  1.8948e+00],
          [-9.5009e-01, -3.3163e-01, -1.8823e-01, -1.0211e+00,  2.1603e-01,
      

In [75]:


# 原始数据
scores = torch.tensor([
    [80, 90, 70],
    [60, 85, 95],
    [75, 88, 82]
], dtype=torch.float32)

weights = torch.tensor([0.3, 0.5, 0.2]).unsqueeze(1)
weights,weights.shape


(tensor([[0.3000],
         [0.5000],
         [0.2000]]),
 torch.Size([3, 1]))